# Objective

Continuation of run 9, decreasing the randomness and increasing far-sightedness. 500 runs per set, 1500 per round for more robust results (9000 games ~ 900-1500 minutes, 15-25 hours)

In [1]:
from game import Game
from ai import SplendorAI
from player import get_phase_parameters
from constants import *
from datetime import datetime
# from player import get_phase_parameters
import sys
from collections import defaultdict
from collections import Counter



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# simply used to initialize players
base_game = Game(id=0, n_players=4, players_compare_self_to_others=True)

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6645655095143375004
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10241700660
locality {
  bus_id: 1
  links {
  }
}
incarnation: 438969804651288513
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
# temperature will decrease over time
def calculate_temperature(round):
    return 1.9/(1 + round/1.2) /(1.5+round/1.1)/(1+round/5)

In [6]:
# constants to define; default
NETWORK_HYPERPARAMETERS = {
    # player input
    'player_funnel_layers': [15,12,10],
    'reserved_funnel_layers': [12,10,],
    'inject_reserved_at_player_funnel_index': 1, # 0 same as input, 1 = as first layer, etc.
    'card_funnel_layers': [12,12,8],
    # game input
    'game_funnel_layers': [15, 12, 10],
    'game_objective_funnel_layers': [10, 8],
    'game_card_funnel_layers': [15, 12, 10],
    # overall, slightly increased from default
    'main_dense_layers': [108, 52, 16], #this is when everything is combined

    # output layers
    # this does not include the win layer
    'output_layers': [
        {
            'name': 'Q1',
            'lag': 1,
            'score': 1,
            'discount': 0.1,
            'gems': 0.01,

        },
        {
            'name': 'Q3',
            'lag':  3,
            'score': 1,
            'discount': 0.1,
            'gems': 0,
        },
        {
            'name': 'Q5',
            'lag': 5,
            'score': 1,
            'discount': 0.05,
            'gems': 0,
        },
    ],
}

def get_phase_parameters(phase):
    """
    training will be divided into 5 phases

    """
    if phase==1:
        return {
            'Q1': 0.5,
            'Q3': 0.3,
            'Q5': 0.15,
            'win': 0.05,
        }
    elif phase==2:
        return {
            'Q1': 0.35,
            'Q3': 0.25,
            'Q5': 0.25,
            'win': 0.15,
        }
    elif phase==3:
        return {
            'Q1': 0.25,
            'Q3': 0.25,
            'Q5': 0.25,
            'win': 0.25,
        }
    elif phase==4:
        return {
            'Q1': 0.15,
            'Q3': 0.2,
            'Q5': 0.35,
            'win': 0.3,
        }
    elif phase==5:
        return {
            'Q1': 0.05,
            'Q3': 0.1,
            'Q5': 0.35,
            'win': 0.50,
        }
    elif phase==6:
        return {
            'Q1': 0.03,
            'Q3': 0.1,
            'Q5': 0.22,
            'win': 0.75
        }

In [7]:
players = base_game.players
# game_data = defaultdict(list)
run_name='run_9'


# load previous models
for i, player in enumerate(players):
    player.ai.load_models('{run_name}_player_%s_'.format(run_name=run_name) % i, 6)




In [8]:
# load data from previous trial, as they are all dictionaries
import pickle
with open('{run_name}_game_data.dat'.format(run_name=run_name), 'rb') as f:
    game_data = pickle.load(f)
    
with open('{run_name}_duration_data.dat'.format(run_name=run_name), 'rb') as f:
    set_durations = pickle.load(f)

In [9]:
def summarize_counters(counter_list):
    master_counter = Counter()
    n = len(counter_list)
    for counter in counter_list:
        master_counter.update(counter)
    keys = list(master_counter.keys())
    for k in sorted(keys):
        master_counter[k] /= n
    return master_counter

In [10]:
# set_durations = {}

# Test multiple runs of games

This should constitute the first half, and get a good idea of how successful these methods are

In [11]:
n_rounds = 6
n_sets_per_round = 3
n_simulations_per_set = 500

start_time = datetime.now()
for i in range(7, 7+n_rounds):
    print('ON ROUND', i)
    phase = min(6, i // 2 + 1)
    temperature = calculate_temperature(i)
    
    for p_i, player in enumerate(players):
        player.temperature = temperature
        player.decision_weighting = get_phase_parameters(phase)
        
    for j in range(n_sets_per_round):
        set_start_time = datetime.now()
        for k in range(n_simulations_per_set):
            # if (i==0) and (j==0):
            #    # soft restart
            #    break
            new_game = Game(id=i*n_sets_per_round+n_simulations_per_set + j*n_simulations_per_set+k, 
                            players=players)
            stalemate = new_game.run()  
            # stalemates should be extraordinarily rare and pretty much nonexistent
            if stalemate: 
                very_long_game = new_game
                print('teaching ai not to stalemate -_-')
                for player in players:
                    player.transfer_history_to_ai()
                    player.ai.train_models(verbose=0,n_epochs=3)
            sys.stdout.write('.')
            sys.stdout.flush()
            # record historic game data for each set
            game_data[(i,j)].append(new_game.copy_plain_data_for_self_and_players())
        set_stop_time = datetime.now()
        duration = (set_start_time-set_stop_time).seconds
        for player in players:
            player.transfer_history_to_ai()
            # train more epochs this time to make better predictions
            player.ai.train_models(verbose=0, n_epochs=30, batch_size=1500)
        set_durations[(i,j,k)] = duration
        print('/')
        sys.stdout.flush()
        avg_game_length = np.mean([x['game']['turn'] for x in game_data[(i,j)]])
        print('R/S %d/%d AVERAGE GAME LENGTH: ' % (i, j), avg_game_length)
        avg_cards_purchased = np.mean([
            [
                pdata['n_cards'] 
                for pdata in x['players'].values()
            ] 
            for x in game_data[(i,j)]
        ]
        )
        print('R/S %d/%d AVERAGE CARDS PURCHASED: ' % (i, j), avg_cards_purchased)
        # calculate win rates
        player_data = [x['players'] for x in game_data[(i,j)]]
        win_values = [[player_data[idx][pid]['win'] for idx in range(n_simulations_per_set)] for pid in range(4)]
        win_rates = [np.mean(v) for v in win_values]
        print('WIN RATES: ', str(win_rates))
        # reservation averages
        reservation_counters = [
            [
                player_data[idx][pid]['total_times_reserved'] 
                for idx in range(n_simulations_per_set)
            ]
            for pid in range(4)
        ]
        print('RESERVATION SUMMARY: ', [summarize_counters(counters) for counters in reservation_counters])
        
        # gem taking averages
        gem_counters = [
            [
                player_data[idx][pid]['total_gems_taken'] 
                for idx in range(n_simulations_per_set)
            ]
            for pid in range(4)
        ]
        print('GEMS SUMMARY: ', [summarize_counters(counters) for counters in gem_counters])
                             
    for player in players:
        player.reset(reset_extended_history=True)
   
    
    for p_i, player in enumerate(players):
        model_name = '{run_name}_player_%s_'.format(run_name=run_name) % str(p_i)
        player.ai.save_models(model_name, index=i)
    
    
stop_time = datetime.now()
for time in [start_time, stop_time]:
    print(time.strftime('%x %X'))
    
# save historic data
import pickle
with open('{run_name}_game_data.dat'.format(run_name=run_name), 'wb') as f:
    pickle.dump(game_data, f)
    
with open('{run_name}_duration_data.dat'.format(run_name=run_name), 'wb') as f:
    pickle.dump(set_durations, f)
    


ON ROUND 7
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................../
R/S 7/0 AVERAGE GAME LENGTH:  125.98039215686275
R/S 7/0 AVERAGE CARDS PURCHASED:  12.916666666666666
WIN RATES:  [0.09, 0.078, 0.478, 0.354]
RESERVATION SUMMARY:  [Counter({(2, 'board'): 1.308, (3, 'board'): 1.286, (1, 'board'): 0.944, (3, 'topdeck'): 0.25, (2, 'topdeck'): 0.248, (1, 'topdeck'): 0.228}), Counter({(3, 'board'): 1.258, (1, 'board'): 1.142, (2, 'board'): 1.112, (3, 'topdeck'): 0.26, (1, 'topdeck'): 0.234, (2, 'topdeck'): 0.226}), Counter({(3, 'board'): 1.818, (2, 'boa

..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................../
R/S 8/2 AVERAGE GAME LENGTH:  125.712
R/S 8/2 AVERAGE CARDS PURCHASED:  13.023
WIN RATES:  [0.11, 0.124, 0.368, 0.404]
RESERVATION SUMMARY:  [Counter({(3, 'board'): 1.342, (2, 'board'): 1.17, (1, 'board'): 1.002, (2, 'topdeck'): 0.252, (3, 'topdeck'): 0.25, (1, 'topdeck'): 0.208}), Counter({(3, 'board'): 1.43, (2, 'board'): 1.144, (1, 'board'): 0.902, (1, 'topdeck'): 0.228, (3, 'topdeck'): 0.224, (2, 'topdeck'): 0.182}), Counter({(3, 'board'): 1.82, (2, 'board'): 1.146, (1, 'board'): 0.608, (2

...............................................................................................................................................................................................................................................................................................................................................................................................................................................................ERROR WITH NUMBER OF ROUNDS
teaching ai not to stalemate -_-
...................................................../
R/S 10/2 AVERAGE GAME LENGTH:  126.832
R/S 10/2 AVERAGE CARDS PURCHASED:  13.0975
WIN RATES:  [0.204, 0.092, 0.296, 0.408]
RESERVATION SUMMARY:  [Counter({(3, 'board'): 1.602, (2, 'board'): 1.114, (1, 'board'): 0.9, (2, 'topdeck'): 0.21, (1, 'topdeck'): 0.17, (3, 'topdeck'): 0.158}), Counter({(3, 'board'): 1.27, (2, 'board'): 1.106, (1, 'board'): 1.106, (3, 'topdeck'): 0.26, (2, 'topdeck'): 0.246, (1, 'topdeck'): 0.194}), Counter({(3

/ntfsl/workspace/splendor_ai/player.py:231: RuntimeWarning: invalid value encountered in true_divide
  p=score/np.sum(score)


..

/ntfsl/workspace/splendor_ai/player.py:231: RuntimeWarning: invalid value encountered in less
  p=score/np.sum(score)


...........................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [12]:
# save historic data
import pickle
with open('{run_name}_game_data.dat'.format(run_name=run_name), 'wb') as f:
    pickle.dump(game_data, f)
    
with open('{run_name}_duration_data.dat'.format(run_name=run_name), 'wb') as f:
    pickle.dump(set_durations, f)

In [ ]:
# players[0].ai.load_models('run6_player_0_round_', 1)

In [ ]:
from collections import Mapping, Container 
from sys import getsizeof

def deep_getsizeof(o, ids): 

    d = deep_getsizeof
    if id(o) in ids:
        return 0

    r = getsizeof(o)
    ids.add(id(o))

    if isinstance(o, str) or isinstance(0, bytes):
        return r
    
    if isinstance(o, np.ndarray):
        return r

    if isinstance(o, Mapping):
        return r + sum(d(k, ids) + d(v, ids) for k, v in o.items())

    if isinstance(o, Container):
        return r + sum(d(x, ids) for x in o)

    return r

deep_getsizeof(game_data, set())
# deep_getsizeof(players[0].extended_serialized_action_history[0], set())

# for k in list(locals().keys()):
#    v = locals()[k]
#    size = deep_getsizeof(v, set())
#    if size > 100000:
#        print(k, ':', size)

In [ ]:
# players[1].reset(reset_extended_history=True)
# for k in dir(players[1]):
#    v = getattr(players[1], k)
#    if isinstance(v, (list, dict)):
#        print(k, len(v))

In [ ]:
#new_game = Game(id=i*200 + j*100+1.1, players=players)

In [ ]:
#for k in dir(players[2]):
#    v = getattr(players[2], k)
#    if isinstance(v, (list, dict)):
#        print(k, len(v))

In [ ]:
#len(players[0].ai.extended_serialized_history)

# NOTES

Runs are complete. Data can be analyzed in a different environment. Tweaks may be made for future runs based on simulation results from loaded models.